In [ ]:
import robust_pdefind as rp
from matplotlib import pyplot as plt
from pdefind import *
import numpy as np
from get_params import *
from tqdm import tqdm

In [ ]:
#we adapt the function compute_params from the get_params file so that it implements the PCA approach of Li et al.
def compute_robust_params(dens_list, dest_mat):  
    for i in tqdm(range(len(dens_list))):
        U = np.transpose(dens_list[i])
        dt=2
        dx =1
        Z, E = rp.RobustPCA(U, lam_2 = 0.035)
        
        ##For loop below re-scales the maximum value of the obtained parameters##
        for j in range(Z.shape[1]):
            Z[:,j] = 0.5*Z[:,j]*(max(U[:,j])/max(Z[:,j]) + np.mean(U[:,j])/np.mean(Z[:,j]))
        
        Ut, R, rhs_des = build_linear_system(Z,dt,dx,D=2,P=2,time_diff = 'poly',deg_x =4)
        w, X, E2 = rp.Robust_LRSTR(R, Ut, rhs_des)
        dest_mat[:,i] = np.real(w[:,0])
    return dest_mat

In [ ]:
FNU1 = np.loadtxt('/scratch/martinaperez/ABM_data/FNU1.txt')
ud1 = convert_to_list(FNU1,1000)
robust_FNU_params = compute_robust_params(ud1,np.empty((9,1000)))
np.savetxt('robust_PCA_coeffs1.txt',robust_FNU_params)